In [ ]:
from __init__ import *

In [ ]:
signal_df_ = []

for index, data in enumerate(SETTING["model_data_"]):

    signal_df = pd.read_csv(
        "../output/signal/{}.tsv".format(index), sep="\t", index_col=0
    )

    signal_df.columns.name = data["axis_1_name"]

    signal_df_.append(signal_df)

In [ ]:
if SETTING["model_mode"] == "deep":

    matrix_factorization_mode = "ws"

    sync_axis = 1

elif SETTING["model_mode"] == "range":

    matrix_factorization_mode = "hs"

    sync_axis = 0

signal_df_ = kraft.pd.sync(signal_df_, sync_axis)

In [ ]:
for r in SETTING["resolution_"]:

    untangle_ = np.asarray(
        tuple(kraft.pd.untangle(signal_df) for signal_df in signal_df_), dtype="object",
    )

    w_, h_, error__ = kraft.matrix_factorization.factorize(
        untangle_[:, 0], matrix_factorization_mode, r
    )

    directory_path = "../output/matrix_factorization_r{}/".format(r)

    kraft.path.path(directory_path)

    kraft.matrix_factorization.plot(
        w_,
        h_,
        untangle_[:, 1],
        untangle_[:, 2],
        untangle_[:, 3],
        untangle_[:, 4],
        error__,
        directory_path=directory_path,
    )

    factor_label_ = kraft.matrix_factorization.make_factor_label_(r)

    w_df_ = tuple(
        kraft.pd.entangle(
            w, untangle_[w_index, 1], factor_label_, untangle_[w_index, 3], "Factor"
        )
        for w_index, w in enumerate(w_)
    )

    h_df_ = tuple(
        kraft.pd.entangle(
            h, factor_label_, untangle_[h_index, 2], "Factor", untangle_[h_index, 4]
        )
        for h_index, h in enumerate(h_)
    )

    file_path_template = "{}{{}}_{{}}.tsv".format(directory_path)

    for w_index, w_df in enumerate(w_df_):

        w_df.to_csv(file_path_template.format("w", w_index), sep="\t")

    for h_index, h_df in enumerate(h_df_):

        h_df.to_csv(file_path_template.format("h", h_index), sep="\t")